In [9]:
#from ratemyprof_api import RateMyProfApi
import pandas as pd

import requests
import json
import math
import csv
import os

from professor import Professor

In [31]:
def get_num_of_professors(id):
    page = requests.get(
        "http://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="
        + str(id)
    )  # get request for page
    temp_jsonpage = json.loads(page.content)
    num_of_prof = (
        temp_jsonpage["remaining"] + 20
    )  # get the number of professors at William Paterson University
    return num_of_prof

In [44]:
def scrape_prof(id: str):
    professors = dict()
    num_of_prof = get_num_of_professors(id)
    num_of_pages = math.ceil(num_of_prof / 20)
    df = pd.DataFrame()

    for i in range(1, num_of_pages + 1):  # the loop insert all professor into list
        page = requests.get(
            "http://www.ratemyprofessors.com/filter/professor/?&page="
            + str(i)
            + "&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="
            + str(id)
        )
        json_response = json.loads(page.content)
        temp_list = json_response["professors"]

        for json_professor in json_response["professors"]:
            #print(json_professor)
            df1 = pd.DataFrame(json_professor, index=[0])
            df = pd.concat([df,df1], ignore_index=True)

            professor = Professor(
                json_professor["tid"],
                json_professor["tFname"],
                json_professor["tLname"],
                json_professor["tNumRatings"],
                json_professor["overall_rating"])

            professors[professor.ratemyprof_id] = professor
    return df

In [49]:
byui = 1754
byu = 135
byuh = 136

In [50]:
byu = scrape_prof(byu)
byui = scrape_prof(byui)
byuh = scrape_prof(byuh)

In [68]:
byu.dropna(subset=['overall_rating'], inplace=True)
byui.dropna(subset=['overall_rating'], inplace=True)
byuh.dropna(subset=['overall_rating'], inplace=True)

In [70]:
byu['overall_rating'] = pd.to_numeric(byu['overall_rating'])
byui['overall_rating'] = pd.to_numeric(byui['overall_rating'])
byuh['overall_rating'] = pd.to_numeric(byuh['overall_rating'])

In [71]:
df = pd.concat([byu,byui,byuh],ignore_index=True)

In [72]:
df = df.drop(['tid', 'rating_class', 'contentType', 'categoryType'], axis=1)

In [73]:
df.tail()

,tDept,tSid,institution_name,tFname,tMiddlename,tLname,tNumRatings,overall_rating
7942,English,136,Brigham Young University - Hawaii,Mark,,Wolfersberger,3,3.7
7943,Information Systems,136,Brigham Young University - Hawaii,Stuart,,Wolthuis,4,4.7
7944,Mathematics,136,Brigham Young University - Hawaii,Ka,Lun,Wong,5,3.8
7945,Languages,136,Brigham Young University - Hawaii,Earl,,Wyman,6,3.7
7946,Communication,136,Brigham Young University - Hawaii,Brent,,Yergensen,10,3.8


In [78]:
with open("df.csv", 'w') as f:
    f.write(df.to_csv(index=False))
with open("byu.csv", 'w') as f:
    f.write(byu.to_csv(index=False))
with open("byui.csv", 'w') as f:
    f.write(byui.to_csv(index=False))
with open("byuh.csv", 'w') as f:
    f.write(byuh.to_csv(index=False))